<a href="https://colab.research.google.com/github/zarah-ml/Car-classification/blob/main/TransferLearning_RESNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Car classification using Transfer Learning**

In [ ]:
!pip3 install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html

## Download dataset 
We are using Stanford car dataset that contains 16,185 images of 196 classes of cars. The data is split into 8,144 training images and 8,041 testing images. The devkit folder includes class labels for training images. 

In [ ]:
!wget http://imagenet.stanford.edu/internal/car196/cars_train.tgz
!wget http://imagenet.stanford.edu/internal/car196/cars_test.tgz
!wget https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
!tar xzf cars_train.tgz
!tar xzf cars_test.tgz
!tar xzf car_devkit.tgz

Mount Google drive to save the trained model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## Define utility functions
Here we define functions to display training and validation losses and accuracy. We also define CarDataset function to preprocess training images and labels to be used by dataloader.

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from scipy import io 
import os
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from tqdm import tqdm
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

matplotlib.style.use('ggplot')

def save_plot(train, valid, plot_name):
    # loss plots
    plt.figure(figsize=(10, 7))
    plt.plot(train, color='orange', label='train_' + plot_name)
    plt.plot(valid, color='red', label='validataion_' + plot_name)
    plt.xlabel('Epochs')
    plt.ylabel(plot_name)
    plt.legend()
    plt.savefig('./' + plot_name + '.jpg')
    plt.show()

def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs==labels)/float(labels.size)

class carDataset(Dataset):
  def __init__(self, image_dir, annotation_file, transform, train = True):
    
    self.img_dir = image_dir
    self.ann_file = annotation_file
    self.transform = transform

    self.img_list = []
    self.trg_list = []

    mat_file = io.loadmat(self.ann_file)

    for idx, img in enumerate(mat_file['annotations'][0]):
              
        path = os.path.join(self.img_dir, img[-1][0])
        img_data = Image.open(path).convert("RGB")
               
        if img[-2][0][0] <=196:
            self.trg_list.append(img[-2][0][0]-1)
            self.img_list.append(img_data)

  def __getitem__(self, index):

    img = self.img_list[index]
    img = self.transform(img)
    target = self.trg_list[index]
    
    return img , target

  def __len__(self):
   return len(self.img_list)
     


## Training and Validation
Here we define functions to train and validate the model at each epoch.

In [ ]:
def train(model, dataloader, dataset, device, optimizer, scheduler, criterion, epoch, path):
    model.train()
    running_loss = 0.0
    batch_acc = 0
    counter = 0
    for idx, (images, labels) in tqdm(enumerate(dataloader)):
        counter += 1
        
        images = images.to(device)
        labels = torch.tensor(labels, dtype=torch.long, device=device)
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        running_loss += loss.item()
        optimizer.step()
       
        labels = labels.data.cpu().numpy()
        outputs = outputs.data.cpu().numpy()
        batch_acc += accuracy(outputs, labels)

        if epoch % 10 == 0:
               torch.save(model.state_dict(), path)

    
    train_loss = running_loss / counter 
    train_acc =  100 * (batch_acc / counter)
    return train_loss, train_acc

def validate(model, dataloader, dataset, device, scheduler, criterion):
    model.eval()
    val_running_loss = 0.0
    val_batch_acc = 0
    counter = 0
    with torch.no_grad():
        for idx, (images, labels) in tqdm(enumerate(dataloader)):
            counter += 1
            
            images = images.to(device)
            labels = torch.tensor(labels, dtype=torch.long, device=device)
                        
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item()
            labels = labels.data.cpu().numpy()
            outputs = outputs.data.cpu().numpy()
            val_batch_acc += accuracy(outputs, labels)     
            
                           
    val_loss = val_running_loss / counter
    val_acc =  100 * (val_batch_acc / counter)
    scheduler.step(val_acc)
    return val_loss, val_acc

## Load Model
Here we create some transforms for our data and load the train/validate data + labels. We load the pretrained Resnet50 model and freeze all layers except the last residual block. We change the output dimensions of the fully connected layer.

In [ ]:
def main(learning_rate, batch_size, epochs):
  
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_transform = transforms.Compose([transforms.Resize((244,244)),
                                       transforms.RandomRotation(30),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
    valid_transform = transforms.Compose([transforms.Resize((244,244)),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    data_ann_path = './devkit/cars_train_annos.mat'
    data_img_path = './cars_train'
    dataset =  carDataset(data_img_path,data_ann_path, train_transform, train= True)

    # splitting our data
    valid_size  = int(0.1 * len(dataset))
    train_size = len(dataset) - valid_size
    trainset, validset = torch.utils.data.random_split(dataset, [train_size, valid_size])

    train_loader = DataLoader( trainset, batch_size=batch_size, shuffle=True )
    valid_loader = DataLoader(validset, batch_size=batch_size, shuffle=False)
    
    # path to save the model
    model_save_name = 'carclassification.pt'
    path = F"/content/gdrive/My Drive/{model_save_name}"

    # set the learning parameters
    hparams = {
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "epochs": epochs,
    }

    # initialize the model
    model = models.resnet34(pretrained=True).to(device)

    num_ftrs = model.fc.in_features
    print(num_ftrs)

    model.fc = nn.Sequential(
               nn.Dropout(0.5),
               nn.Linear(num_ftrs, 196)).to(device)   
    
    #print(model.parameters)
    #for name, param in model.named_parameters():
        #print(name,param.requires_grad)

    optimizer = optim.SGD(model.parameters(), lr=hparams['learning_rate'], momentum=0.9)
    criterion = nn.CrossEntropyLoss()
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)

    # a list to save all the train and validation loss 
    train_loss = []
    valid_loss = []
    train_acc = []
    valid_acc = []

    for epoch in range(epochs):
        print(f"Epoch {epoch+1} of {epochs}")
        train_epoch_loss, train_epoch_acc = train(model, train_loader, trainset, device, optimizer,  scheduler, criterion, epoch, path)
        valid_epoch_loss, valid_epoch_acc = validate(model, valid_loader, validset, device,  scheduler, criterion)

        train_loss.append(train_epoch_loss)
        valid_loss.append(valid_epoch_loss)
        train_acc.append(train_epoch_acc)
        valid_acc.append(valid_epoch_acc)

        
        print(f"Train Loss: {train_epoch_loss:.4f}")
        print(f"Val Loss: {valid_epoch_loss:.4f}")
        print(f"Train accuracy: {train_epoch_acc:.4f}")
        print(f"Val accuracy: {valid_epoch_acc:.4f}")

   
    # save the loss plots to disk
    save_plot(train_loss, valid_loss, "loss")
    save_plot(train_acc, valid_acc, "accuracy")

    print('Training Complete!!!')

In [ ]:
learning_rate = 0.01
batch_size = 64
epochs = 10


main(learning_rate, batch_size, epochs)